## Connect to DB

In [468]:
%%configure -f
{ "name":"Spark-to-Cosmos_DB_Connector",  
  "conf": {
    "spark.jars.packages": "com.microsoft.azure:azure-cosmosdb-spark_2.4.0_2.11:1.4.0",
    "spark.jars.excludes": "org.scala-lang:scala-reflect"
   }
}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1574361531834_0005,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1574361531834_0005,pyspark,idle,Link,Link,✔


## Get Data from containers

In [469]:
# Champion Container
readConfig = {
"Endpoint" : "https://lolpredict.documents.azure.com:443/",
"Masterkey" : "UcLcT6bvFUsk677XfibNhJIIqiAstRkaWWrqTJaQOCmVqx0IN0J1ufYdpbsarowutD6nMSKaaLS1NqcD0QDIdQ==",
"Database" : "league_all_data",
"Collection" : "champions", 
}
champ = spark.read.format("com.microsoft.azure.cosmosdb.spark").options(**readConfig).load()

In [470]:
# match container
matchConfig = {
"Endpoint" : "https://lolpredict.documents.azure.com:443/",
"Masterkey" : "UcLcT6bvFUsk677XfibNhJIIqiAstRkaWWrqTJaQOCmVqx0IN0J1ufYdpbsarowutD6nMSKaaLS1NqcD0QDIdQ==",
"Database" : "league_all_data",
"Collection" : "matches", 
}

match = spark.read.format("com.microsoft.azure.cosmosdb.spark").options(**matchConfig).load()

In [471]:
#match filter
match.createOrReplaceTempView("table1")
srmatch=spark.sql("SELECT * from table1 where stats.gameMode='CLASSIC' and stats.gameDuration>420") #filtered match table

In [472]:
#player container
playConfig = {
"Endpoint" : "https://lolpredict.documents.azure.com:443/",
"Masterkey" : "UcLcT6bvFUsk677XfibNhJIIqiAstRkaWWrqTJaQOCmVqx0IN0J1ufYdpbsarowutD6nMSKaaLS1NqcD0QDIdQ==",
"Database" : "league_all_data",
"Collection" : "players", 
}

play = spark.read.format("com.microsoft.azure.cosmosdb.spark").options(**playConfig).load()

In [473]:
#player filtering/transforming
import pyspark.sql.functions as f
play.createOrReplaceTempView("table2")
play2=spark.sql("SELECT mastery, id from table2")
play2.show(1)

+--------------------+--------------------+
|             mastery|                  id|
+--------------------+--------------------+
|[31119, 218, 2712...|EuXHkripW2Qav3H2l...|
+--------------------+--------------------+
only showing top 1 row

In [474]:
#separates k/v pairs into many columns
expdic2=play2.select(f.col('id').alias('summonerID'),
                     f.col('mastery.*'))
expdic2.show(1)

+--------------------+-----+---+----+----+----+----+----+----+----+----+----+----+----+----+---+---+-----+---+----+----+----+----+----+----+-----+----+---+---+---+----+----+----+----+-----+----+----+----+----+-----+----+----+----+---+----+----+---+----+---+----+----+----+----+-----+----+----+----+----+----+-----+----+-----+-----+----+----+----+-----+-----+---+----+----+----+----+----+----+---+----+----+----+----+----+---+-----+-----+----+-----+----+---+----+----+---+----+---+-----+---+---+----+----+----+---+---+----+----+----+----+----+---+----+----+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+---+----+----+---+----+---+----+----+----+---+----+----+----+----+---+---+----+----+----+----+----+----+----+
|          summonerID|    1| 10| 101| 102| 103| 104| 105| 106| 107|  11| 110| 111| 112| 113|114|115|  117|119|  12| 120| 121| 122| 126| 127|   13| 131|133|134|136|  14| 141| 142| 143|  145|  15| 150| 154| 157|   16| 161| 163| 164| 17|  18|  19|  2|  20|201| 20

## Transforming into final mastery table

In [475]:
from pyspark.sql.functions import array, col, explode, struct, lit
def flat_col(df, by):

    # Filter dtypes and split into column names and type description
    cols, dtypes = zip(*((c, t) for (c, t) in df.dtypes if c not in by))
    # Spark SQL supports only homogeneous columns
    assert len(set(dtypes)) == 1, "All columns have to be of the same type"

    # Create and explode an array of (column_name, column_value) structs
    kvs = explode(array([
      struct(lit(c).alias("championId"), col(c).alias("points")) for c in cols
    ])).alias("kvs")

    return df.select(by + [kvs]).select(by + ["kvs.championId", "kvs.points"])

champ_mastery=flat_col(expdic2, ["summonerID"])
champ_mastery.show(5)

+--------------------+----------+------+
|          summonerID|championId|points|
+--------------------+----------+------+
|EuXHkripW2Qav3H2l...|         1| 31119|
|EuXHkripW2Qav3H2l...|        10|   218|
|EuXHkripW2Qav3H2l...|       101|  2712|
|EuXHkripW2Qav3H2l...|       102|  null|
|EuXHkripW2Qav3H2l...|       103|  5543|
+--------------------+----------+------+
only showing top 5 rows

## Champion Table

In [476]:
champ.createOrReplaceTempView("table3")
champ2=spark.sql("SELECT id, name, stats.armorperlevel, stats.attackdamageperlevel, stats.attackspeed, stats.attackrange, stats.spellblockperlevel, stats.hpperlevel FROM table3")
champ2.collect()

[Row(id=u'266', name=u'Aatrox', armorperlevel=3.25, attackdamageperlevel=5.0, attackspeed=0.651, attackrange=175, spellblockperlevel=1.25, hpperlevel=90), Row(id=u'103', name=u'Ahri', armorperlevel=3.5, attackdamageperlevel=3.0, attackspeed=0.668, attackrange=550, spellblockperlevel=0.5, hpperlevel=92), Row(id=u'84', name=u'Akali', armorperlevel=3.5, attackdamageperlevel=3.3, attackspeed=0.625, attackrange=125, spellblockperlevel=1.25, hpperlevel=95), Row(id=u'12', name=u'Alistar', armorperlevel=3.5, attackdamageperlevel=3.62, attackspeed=0.625, attackrange=125, spellblockperlevel=1.25, hpperlevel=106), Row(id=u'32', name=u'Amumu', armorperlevel=3.8, attackdamageperlevel=3.8, attackspeed=0.736, attackrange=125, spellblockperlevel=1.25, hpperlevel=84), Row(id=u'34', name=u'Anivia', armorperlevel=4.0, attackdamageperlevel=3.2, attackspeed=0.625, attackrange=600, spellblockperlevel=0.5, hpperlevel=82), Row(id=u'1', name=u'Annie', armorperlevel=4.0, attackdamageperlevel=2.625, attackspeed=

## Join champ and mastery tables

In [477]:
mast_champ_join = champ_mastery.join(champ2, champ_mastery.championId == champ2.id)
mast_champ_join.show(2)

+--------------------+----------+------+---+-------+-------------+--------------------+-----------+-----------+------------------+----------+
|          summonerID|championId|points| id|   name|armorperlevel|attackdamageperlevel|attackspeed|attackrange|spellblockperlevel|hpperlevel|
+--------------------+----------+------+---+-------+-------------+--------------------+-----------+-----------+------------------+----------+
|EuXHkripW2Qav3H2l...|        51|  5333| 51|Caitlyn|          3.5|                2.88|      0.625|        650|               0.5|        91|
|RBQZJQA6Acz3VqBqd...|        51| 30573| 51|Caitlyn|          3.5|                2.88|      0.625|        650|               0.5|        91|
+--------------------+----------+------+---+-------+-------------+--------------------+-----------+-----------+------------------+----------+
only showing top 2 rows

## Get label vector

In [649]:
import pyspark
label2 = srmatch.select(pyspark.sql.functions.col('id'),
                       pyspark.sql.functions.col('stats.teams.teamId').alias('teamId'),
                       pyspark.sql.functions.col('stats.teams.win').alias('win'))

In [650]:
from pyspark.sql.types import IntegerType, ArrayType
from pyspark.sql.functions import udf
def create_label(row):
    if row[0]=='Win':
        return 1
    return 0
label_udf = udf(create_label, IntegerType())
label2 = label2.withColumn('label', label_udf(label2.win))
label2.show(10)

+----------+----------+-----------+-----+
|        id|    teamId|        win|label|
+----------+----------+-----------+-----+
|3207597356|[100, 200]|[Fail, Win]|    0|
|3207612018|[100, 200]|[Fail, Win]|    0|
|3206625787|[100, 200]|[Fail, Win]|    0|
|3206567503|[100, 200]|[Win, Fail]|    1|
|3204655759|[100, 200]|[Fail, Win]|    0|
|3204671034|[100, 200]|[Win, Fail]|    1|
|3204607354|[100, 200]|[Fail, Win]|    0|
|3204573746|[100, 200]|[Win, Fail]|    1|
|3207798479|[100, 200]|[Win, Fail]|    1|
|3207820941|[100, 200]|[Win, Fail]|    1|
+----------+----------+-----------+-----+
only showing top 10 rows

In [651]:
label_col = label2.drop('teamId','win')
label_col.show(2)

+----------+-----+
|        id|label|
+----------+-----+
|3207597356|    0|
|3207612018|    0|
+----------+-----+
only showing top 2 rows

## Save to CosmosDB

In [16]:
writeConfig = {
    "Endpoint": "https://lolpredict.documents.azure.com:443/",
    "Masterkey": "UcLcT6bvFUsk677XfibNhJIIqiAstRkaWWrqTJaQOCmVqx0IN0J1ufYdpbsarowutD6nMSKaaLS1NqcD0QDIdQ==",
    "Database": "league_extraction",
    "Collection": "champion_table",
    "Upsert": "true"
}

# Write to Cosmos DB from the flights DataFrame
champ2.write.format("com.microsoft.azure.cosmosdb.spark").options(
    **writeConfig).save()

## Create stats table

### Match_Summoner exploded table

In [481]:
matches_view =spark.sql("SELECT id, matches FROM table2")
match_sum_tab=matches_view.withColumn('matchId', explode('matches'))
match_sum_tab.show(3)

+--------------------+--------------------+----------+
|                  id|             matches|   matchId|
+--------------------+--------------------+----------+
|EuXHkripW2Qav3H2l...|[3207690957, 3207...|3207690957|
|EuXHkripW2Qav3H2l...|[3207690957, 3207...|3207597356|
|EuXHkripW2Qav3H2l...|[3207690957, 3207...|3207612018|
+--------------------+--------------------+----------+
only showing top 3 rows

### Match_summoner_stats table

In [482]:
import numpy as np
from tqdm import tqdm_notebook

In [483]:
matches_ids = srmatch.select("id")
matchIds_list = np.array(matches_ids.collect())
matchIds_list = [str(i[0]) for i in matchIds_list] #ids of all matches 

In [484]:
matchDur =  srmatch.select("stats.gameDuration")
matchDur_list = np.array(matchDur.collect())
matchDur_list = [float(i[0])/60 for i in matchDur_list] #game length for all matches in minutes 

In [485]:
all_matches_stats = srmatch.select(
    'stats.participantIdentities.player.summonerId',
                     'stats.participants.championId',
                     'stats.participants.timeline.lane',
                     'stats.participants.timeline.role',
                    'stats.participants.teamId',
                    'stats.participants.stats.goldEarned',
                    'stats.participants.stats.kills',
                    'stats.participants.stats.assists',
                    'stats.participants.stats.deaths',
                    'stats.participants.stats.visionScore',
                    'stats.participants.stats.totalDamageDealt',
                    'stats.participants.stats.totalDamageTaken',
                    'stats.participants.stats.totalMinionsKilled',
                    'stats.participants.stats.totalTimeCrowdControlDealt')

#stats of all matches
stats_np = np.array(all_matches_stats.collect())

In [486]:
all_match_dict = []
for match,stats,game_length in zip(matchIds_list,stats_np,matchDur_list):
    #combine match stats with stats id to create a new dataframe
    match_dict = {}
    match_dict['match_id']= match
    match_dict['match_length'] = game_length
    summonerIds = stats[:][0]
    champions = stats[:][1]
    lanes = stats[:][2]
    roles = stats[:][3]
    teamId = stats[:][4]
    avg_stats = stats[:][5:]
    avg_stats_dice = [list(i) for i in zip(*avg_stats)]
    match_dict['agg_stats'] = [list(map(float,i)) for i in avg_stats_dice]
    match_dict['lanes'] = [str(i) for i in lanes]
    match_dict['roles'] = [str(i) for i in roles]
    match_dict['teamId'] = [str(i) for i in teamId]
    match_dict['summonerIds'] = [str(i) for i in summonerIds]
    match_dict['champions'] = [str(i) for i in champions]
    all_match_dict.append(match_dict)


In [487]:
match_sum_stats_agg_df = spark.createDataFrame(all_match_dict)
match_sum_stats_agg_df.show(2)

+--------------------+--------------------+--------------------+----------+------------------+--------------------+--------------------+--------------------+
|           agg_stats|           champions|               lanes|  match_id|      match_length|               roles|         summonerIds|              teamId|
+--------------------+--------------------+--------------------+----------+------------------+--------------------+--------------------+--------------------+
|[[7864.0, 3.0, 3....|[245, 13, 114, 25...|[JUNGLE, TOP, MID...|3207597356|             22.55|[NONE, SOLO, SOLO...|[SAim3CJlYb3gTey9...|[100, 100, 100, 1...|
|[[6933.0, 1.0, 8....|[111, 145, 107, 1...|[BOTTOM, BOTTOM, ...|3207612018|27.166666666666668|[DUO_SUPPORT, DUO...|[CInmKwfaqk85iX9G...|[100, 100, 100, 1...|
+--------------------+--------------------+--------------------+----------+------------------+--------------------+--------------------+--------------------+
only showing top 2 rows

/usr/hdp/current/spark2-cli

In [617]:
from pyspark.sql.types import *
from pyspark.sql.functions import col, udf, explode

zip_ = udf(
  lambda a,b,c,d,e,f: list(zip(a,b,c,d,e,f)),
  ArrayType(StructType([
      # Adjust types to reflect data types
      StructField("stats", ArrayType(DoubleType())),
      StructField("champion", StringType()),
      StructField("summonerIds", StringType()),
    StructField("lane", StringType()),
    StructField("role", StringType()),
    StructField("team", StringType()),           
  ]))
)

In [618]:
#explode table to create a more fine grain accessing method 
match_sum_stats_s = match_sum_stats_agg_df.withColumn("temp",zip_("agg_stats","champions","summonerIds","lanes","roles","teamId"))
match_sum_stats_s = match_sum_stats_s.withColumn("temp", explode("temp"))

In [678]:
match_sum_stats = match_sum_stats_s.select("match_id",col("temp.stats").alias("player_stats_inv"), 
                         col("temp.champion").alias("champion_inv"),col("temp.summonerIds").alias("summonerId_inv"),
                         col("temp.lane").alias("lanes_inv"),
                         col("temp.role").alias("roles_inv"),
                         col("temp.team").alias("team"),"match_length")
match_sum_stats.show(10)

+----------+--------------------+------------+--------------------+---------+-----------+----+------------+
|  match_id|    player_stats_inv|champion_inv|      summonerId_inv|lanes_inv|  roles_inv|team|match_length|
+----------+--------------------+------------+--------------------+---------+-----------+----+------------+
|3207597356|[7864.0, 3.0, 3.0...|         245|SAim3CJlYb3gTey9H...|   JUNGLE|       NONE| 100|       22.55|
|3207597356|[6533.0, 2.0, 3.0...|          13|EuXHkripW2Qav3H2l...|      TOP|       SOLO| 100|       22.55|
|3207597356|[6526.0, 0.0, 0.0...|         114|FnmTWvNqTGk-2pnJ9...|   MIDDLE|       SOLO| 100|       22.55|
|3207597356|[6823.0, 4.0, 3.0...|          25|9m3TRvR-RpcGTN0We...|   BOTTOM|DUO_SUPPORT| 100|       22.55|
|3207597356|[6492.0, 1.0, 4.0...|          22|nZuXoApHlr15y9--k...|   BOTTOM|  DUO_CARRY| 100|       22.55|
|3207597356|[9405.0, 1.0, 5.0...|          74|AKrjxV60ReBpP9ZIG...|   BOTTOM|  DUO_CARRY| 200|       22.55|
|3207597356|[9522.0, 5.0, 4.

#### clean role table

In [679]:
def processed_role(lane,role,team):
    final_role =""
    if lane != "BOTTOM":
        final_role = "{}_{}".format(team,lane)
    else:
        if "SUPPORT" in role:
            final_role = "{}_SUPPORT".format(team)
        elif "CARRY" in role:
            final_role = "{}_CARRY".format(team)
        else:
            final_role = "CANT"
    if "NONE" in final_role:
        final_role = "CANT"
    return final_role
udf_clean_role = zip_ = udf(
  lambda lane,role,team: processed_role(lane,role,team), StringType()
)
match_sum_stats = match_sum_stats.withColumn('final_role', udf_clean_role("lanes_inv", "roles_inv","team"))
match_sum_stats = match_sum_stats.drop("lanes_inv","roles_inv","team")
match_sum_stats = match_sum_stats.select("*").filter(match_sum_stats.final_role != "CANT") #remove players without clear role
match_sum_stats.show(10)

+----------+--------------------+------------+--------------------+------------+-----------+
|  match_id|    player_stats_inv|champion_inv|      summonerId_inv|match_length| final_role|
+----------+--------------------+------------+--------------------+------------+-----------+
|3207597356|[7864.0, 3.0, 3.0...|         245|SAim3CJlYb3gTey9H...|       22.55| 100_JUNGLE|
|3207597356|[6533.0, 2.0, 3.0...|          13|EuXHkripW2Qav3H2l...|       22.55|    100_TOP|
|3207597356|[6526.0, 0.0, 0.0...|         114|FnmTWvNqTGk-2pnJ9...|       22.55| 100_MIDDLE|
|3207597356|[6823.0, 4.0, 3.0...|          25|9m3TRvR-RpcGTN0We...|       22.55|100_SUPPORT|
|3207597356|[6492.0, 1.0, 4.0...|          22|nZuXoApHlr15y9--k...|       22.55|  100_CARRY|
|3207597356|[9405.0, 1.0, 5.0...|          74|AKrjxV60ReBpP9ZIG...|       22.55|  200_CARRY|
|3207597356|[9522.0, 5.0, 4.0...|          38|abPiRq_oHl1pdzEzJ...|       22.55| 200_MIDDLE|
|3207597356|[10890.0, 10.0, 5...|          29|ffJNX6IZF2NfHxihw...|   

### normalize stats

In [680]:
def normalize_stats(stats,game_length):
    return [ float(stat/game_length) for stat in stats]
    
normalize_udf = udf(normalize_stats,ArrayType(FloatType()))
match_sum_stats_2 = match_sum_stats.withColumn('normalize_stats', normalize_udf(match_sum_stats.player_stats_inv, match_sum_stats.match_length))

In [681]:
match_sum_stats_final = match_sum_stats_2.drop("player_stats_inv") #drop non-normalize stats => we only consider stats/minute

In [682]:
match_sum_stats_final.show(2)

+----------+------------+--------------------+------------+----------+--------------------+
|  match_id|champion_inv|      summonerId_inv|match_length|final_role|     normalize_stats|
+----------+------------+--------------------+------------+----------+--------------------+
|3207597356|         245|SAim3CJlYb3gTey9H...|       22.55|100_JUNGLE|[348.73615, 0.133...|
|3207597356|          13|EuXHkripW2Qav3H2l...|       22.55|   100_TOP|[289.71176, 0.088...|
+----------+------------+--------------------+------------+----------+--------------------+
only showing top 2 rows

## Explode players stats 

In [683]:
match_sum_stats_final = match_sum_stats_final.select("match_id","champion_inv","summonerId_inv","final_role",
                                             match_sum_stats_final.normalize_stats[0].alias("gold_earned"),
                                             match_sum_stats_final.normalize_stats[1].alias("kills"),
                                             match_sum_stats_final.normalize_stats[2].alias("assists"),
                                             match_sum_stats_final.normalize_stats[3].alias("deaths"),
                                             match_sum_stats_final.normalize_stats[4].alias("visionScore"),
                                             match_sum_stats_final.normalize_stats[5].alias("totalDamageDealt"),
                                             match_sum_stats_final.normalize_stats[6].alias("totalDamageTaken"),
                                             match_sum_stats_final.normalize_stats[7].alias("totalMinionsKilled"),
                                             match_sum_stats_final.normalize_stats[8].alias("totalTimeCrowdControlDealt"))

In [684]:
match_sum_stats_final.show(1)

+----------+------------+--------------------+----------+-----------+---------+---------+----------+-----------+----------------+----------------+------------------+--------------------------+
|  match_id|champion_inv|      summonerId_inv|final_role|gold_earned|    kills|  assists|    deaths|visionScore|totalDamageDealt|totalDamageTaken|totalMinionsKilled|totalTimeCrowdControlDealt|
+----------+------------+--------------------+----------+-----------+---------+---------+----------+-----------+----------------+----------------+------------------+--------------------------+
|3207597356|         245|SAim3CJlYb3gTey9H...|100_JUNGLE|  348.73615|0.1330377|0.1330377|0.17738359|  0.6651885|        4702.173|        916.9845|          2.217295|                 17.560976|
+----------+------------+--------------------+----------+-----------+---------+---------+----------+-----------+----------------+----------------+------------------+--------------------------+
only showing top 1 row

## Create average stats table

In [685]:
avg_stats = match_sum_stats_final.select("summonerId_inv","gold_earned","kills",                      
             "assists","deaths","visionScore","totalDamageDealt","totalDamageTaken","totalMinionsKilled",
            "totalTimeCrowdControlDealt")


In [686]:
avg_stats.show(2)

+--------------------+-----------+----------+---------+----------+-----------+----------------+----------------+------------------+--------------------------+
|      summonerId_inv|gold_earned|     kills|  assists|    deaths|visionScore|totalDamageDealt|totalDamageTaken|totalMinionsKilled|totalTimeCrowdControlDealt|
+--------------------+-----------+----------+---------+----------+-----------+----------------+----------------+------------------+--------------------------+
|SAim3CJlYb3gTey9H...|  348.73615| 0.1330377|0.1330377|0.17738359|  0.6651885|        4702.173|        916.9845|          2.217295|                 17.560976|
|EuXHkripW2Qav3H2l...|  289.71176|0.08869179|0.1330377| 0.3104213|        0.0|        3538.182|       1059.0687|         5.4101996|                  3.015521|
+--------------------+-----------+----------+---------+----------+-----------+----------------+----------------+------------------+--------------------------+
only showing top 2 rows

In [687]:
import pyspark.sql.functions as F
summoner_avg_stats = avg_stats.groupBy('summonerId_inv').agg(F.avg('gold_earned'),F.avg('kills'),F.avg("deaths"),
                                     F.avg('assists'),F.avg('visionScore'),F.avg('totalDamageDealt'),
                                     F.avg('totalDamageTaken'),F.avg('totalMinionsKilled'),
                                     F.avg('totalTimeCrowdControlDealt'))

In [688]:
summoner_avg_stats.show(1)

+--------------------+-----------------+------------------+-------------------+-------------------+------------------+---------------------+---------------------+-----------------------+-------------------------------+
|      summonerId_inv| avg(gold_earned)|        avg(kills)|        avg(deaths)|       avg(assists)|  avg(visionScore)|avg(totalDamageDealt)|avg(totalDamageTaken)|avg(totalMinionsKilled)|avg(totalTimeCrowdControlDealt)|
+--------------------+-----------------+------------------+-------------------+-------------------+------------------+---------------------+---------------------+-----------------------+-------------------------------+
|2BaqJJd_u0kzZKntZ...|429.7740042550223|0.2212391985314233|0.21354059129953384|0.26571462196963175|0.5551702507904598|    4901.844656808035|    605.1386195591518|      6.060190337044852|              8.104816675186157|
+--------------------+-----------------+------------------+-------------------+-------------------+------------------+------

### summonerID_avgstats_matchId table

In [689]:
mast_champ_join.show(2)

+--------------------+----------+------+---+-------+-------------+--------------------+-----------+-----------+------------------+----------+
|          summonerID|championId|points| id|   name|armorperlevel|attackdamageperlevel|attackspeed|attackrange|spellblockperlevel|hpperlevel|
+--------------------+----------+------+---+-------+-------------+--------------------+-----------+-----------+------------------+----------+
|EuXHkripW2Qav3H2l...|        51|  5333| 51|Caitlyn|          3.5|                2.88|      0.625|        650|               0.5|        91|
|RBQZJQA6Acz3VqBqd...|        51| 30573| 51|Caitlyn|          3.5|                2.88|      0.625|        650|               0.5|        91|
+--------------------+----------+------+---+-------+-------------+--------------------+-----------+-----------+------------------+----------+
only showing top 2 rows

In [690]:
#join player average stats with player champion mastery and champion stats 
player_champ_stats = summoner_avg_stats.join(mast_champ_join, [
                                                           summoner_avg_stats.summonerId_inv==mast_champ_join.summonerID
                                                            ])

In [691]:
player_champ_stats = player_champ_stats.drop("id","summonerId_inv")
player_champ_stats.show(2)

+-----------------+------------------+-------------------+-------------------+------------------+---------------------+---------------------+-----------------------+-------------------------------+--------------------+----------+------+-------+-------------+--------------------+-----------+-----------+------------------+----------+
| avg(gold_earned)|        avg(kills)|        avg(deaths)|       avg(assists)|  avg(visionScore)|avg(totalDamageDealt)|avg(totalDamageTaken)|avg(totalMinionsKilled)|avg(totalTimeCrowdControlDealt)|          summonerID|championId|points|   name|armorperlevel|attackdamageperlevel|attackspeed|attackrange|spellblockperlevel|hpperlevel|
+-----------------+------------------+-------------------+-------------------+------------------+---------------------+---------------------+-----------------------+-------------------------------+--------------------+----------+------+-------+-------------+--------------------+-----------+-----------+------------------+----------

In [704]:
player_champ_stats.printSchema()

root
 |-- avg(gold_earned): double (nullable = true)
 |-- avg(kills): double (nullable = true)
 |-- avg(deaths): double (nullable = true)
 |-- avg(assists): double (nullable = true)
 |-- avg(visionScore): double (nullable = true)
 |-- avg(totalDamageDealt): double (nullable = true)
 |-- avg(totalDamageTaken): double (nullable = true)
 |-- avg(totalMinionsKilled): double (nullable = true)
 |-- avg(totalTimeCrowdControlDealt): double (nullable = true)
 |-- summonerID: string (nullable = true)
 |-- championId: string (nullable = false)
 |-- points: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- armorperlevel: double (nullable = true)
 |-- attackdamageperlevel: double (nullable = true)
 |-- attackspeed: double (nullable = true)
 |-- attackrange: integer (nullable = true)
 |-- spellblockperlevel: double (nullable = true)
 |-- hpperlevel: integer (nullable = true)

In [705]:
match_sum_stats_join = match_sum_stats_final.select("summonerId_inv","match_id","champion_inv","final_role")
match_sum_stats_join.show(2)

+--------------------+----------+------------+----------+
|      summonerId_inv|  match_id|champion_inv|final_role|
+--------------------+----------+------------+----------+
|SAim3CJlYb3gTey9H...|3207597356|         245|100_JUNGLE|
|EuXHkripW2Qav3H2l...|3207597356|          13|   100_TOP|
+--------------------+----------+------------+----------+
only showing top 2 rows

#### join the final features table for each individual for each match

In [706]:
player_stats_final = match_sum_stats_join.join(player_champ_stats,[
        match_sum_stats_join.summonerId_inv == player_champ_stats.summonerID,
        match_sum_stats_join.champion_inv == player_champ_stats.championId
    ])

In [707]:
player_stats_final = player_stats_final.drop("summonerId_inv","champion_inv")
player_stats_final.printSchema()

root
 |-- match_id: string (nullable = true)
 |-- final_role: string (nullable = true)
 |-- avg(gold_earned): double (nullable = true)
 |-- avg(kills): double (nullable = true)
 |-- avg(deaths): double (nullable = true)
 |-- avg(assists): double (nullable = true)
 |-- avg(visionScore): double (nullable = true)
 |-- avg(totalDamageDealt): double (nullable = true)
 |-- avg(totalDamageTaken): double (nullable = true)
 |-- avg(totalMinionsKilled): double (nullable = true)
 |-- avg(totalTimeCrowdControlDealt): double (nullable = true)
 |-- summonerID: string (nullable = true)
 |-- championId: string (nullable = false)
 |-- points: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- armorperlevel: double (nullable = true)
 |-- attackdamageperlevel: double (nullable = true)
 |-- attackspeed: double (nullable = true)
 |-- attackrange: integer (nullable = true)
 |-- spellblockperlevel: double (nullable = true)
 |-- hpperlevel: integer (nullable = true)

In [708]:
match_with_data = player_stats_final.groupby('match_id').count().select("match_id",F.col('count').alias('number_of_players'))

In [709]:
#join with current all players stats player

In [710]:
players_matchs_stats = player_stats_final.join(match_with_data,[match_with_data.match_id == player_stats_final.match_id]).drop(player_stats_final.match_id)

In [711]:
players_matchs_stats = players_matchs_stats.drop("match_with_data.match_id")

In [712]:
players_matchs_stats.printSchema()

root
 |-- final_role: string (nullable = true)
 |-- avg(gold_earned): double (nullable = true)
 |-- avg(kills): double (nullable = true)
 |-- avg(deaths): double (nullable = true)
 |-- avg(assists): double (nullable = true)
 |-- avg(visionScore): double (nullable = true)
 |-- avg(totalDamageDealt): double (nullable = true)
 |-- avg(totalDamageTaken): double (nullable = true)
 |-- avg(totalMinionsKilled): double (nullable = true)
 |-- avg(totalTimeCrowdControlDealt): double (nullable = true)
 |-- summonerID: string (nullable = true)
 |-- championId: string (nullable = false)
 |-- points: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- armorperlevel: double (nullable = true)
 |-- attackdamageperlevel: double (nullable = true)
 |-- attackspeed: double (nullable = true)
 |-- attackrange: integer (nullable = true)
 |-- spellblockperlevel: double (nullable = true)
 |-- hpperlevel: integer (nullable = true)
 |-- match_id: string (nullable = true)
 |-- number_of_players: lon

In [713]:
players_matchs_stats.show(1)

+----------+----------------+-------------------+------------------+-------------------+------------------+---------------------+---------------------+-----------------------+-------------------------------+--------------------+----------+------+----+-------------+--------------------+-----------+-----------+------------------+----------+----------+-----------------+
|final_role|avg(gold_earned)|         avg(kills)|       avg(deaths)|       avg(assists)|  avg(visionScore)|avg(totalDamageDealt)|avg(totalDamageTaken)|avg(totalMinionsKilled)|avg(totalTimeCrowdControlDealt)|          summonerID|championId|points|name|armorperlevel|attackdamageperlevel|attackspeed|attackrange|spellblockperlevel|hpperlevel|  match_id|number_of_players|
+----------+----------------+-------------------+------------------+-------------------+------------------+---------------------+---------------------+-----------------------+-------------------------------+--------------------+----------+------+----+---------

### remove matches with less than 10 players data 


In [720]:
players_matchs_stats = players_matchs_stats.filter(players_matchs_stats.number_of_players == 10)
players_matchs_stats.count()

350

In [721]:
players_matchs_stats.show(2)

+-----------+------------------+-------------------+-------------------+-------------------+------------------+---------------------+---------------------+-----------------------+-------------------------------+--------------------+----------+------+------+-------------+--------------------+-----------+-----------+------------------+----------+----------+-----------------+
| final_role|  avg(gold_earned)|         avg(kills)|        avg(deaths)|       avg(assists)|  avg(visionScore)|avg(totalDamageDealt)|avg(totalDamageTaken)|avg(totalMinionsKilled)|avg(totalTimeCrowdControlDealt)|          summonerID|championId|points|  name|armorperlevel|attackdamageperlevel|attackspeed|attackrange|spellblockperlevel|hpperlevel|  match_id|number_of_players|
+-----------+------------------+-------------------+-------------------+-------------------+------------------+---------------------+---------------------+-----------------------+-------------------------------+--------------------+----------+-----

## join with label table

In [722]:
label_col.show(1)

+----------+-----+
|        id|label|
+----------+-----+
|3207597356|    0|
+----------+-----+
only showing top 1 row

In [723]:
# label = 100_win or not
players_stats_sep_table = players_matchs_stats.join(label_col,[label_col.id == players_matchs_stats.match_id])

In [724]:
players_stats_sep_table.show(1)

+-----------+------------------+-------------------+-------------------+------------------+-----------------+---------------------+---------------------+-----------------------+-------------------------------+--------------------+----------+------+------+-------------+--------------------+-----------+-----------+------------------+----------+----------+-----------------+----------+-----+
| final_role|  avg(gold_earned)|         avg(kills)|        avg(deaths)|      avg(assists)| avg(visionScore)|avg(totalDamageDealt)|avg(totalDamageTaken)|avg(totalMinionsKilled)|avg(totalTimeCrowdControlDealt)|          summonerID|championId|points|  name|armorperlevel|attackdamageperlevel|attackspeed|attackrange|spellblockperlevel|hpperlevel|  match_id|number_of_players|        id|label|
+-----------+------------------+-------------------+-------------------+------------------+-----------------+---------------------+---------------------+-----------------------+-------------------------------+---------

In [793]:
all_stats_matchs_pd = players_stats_sep_table.toPandas()

# Create feature matrix

In [796]:
import pandas as pd

In [798]:
all_stats_matchs_pd

      final_role  avg(gold_earned)  avg(kills)  avg(deaths)  avg(assists)  \
0    200_SUPPORT        314.184848    0.099776     0.183198      0.515648   
1        100_TOP        395.916968    0.181272     0.160865      0.281782   
2     100_JUNGLE        433.532016    0.247466     0.254786      0.252154   
3    100_SUPPORT        380.765468    0.233464     0.244362      0.270928   
4        200_TOP        375.776585    0.109713     0.211956      0.184976   
5      200_CARRY        482.309204    0.231569     0.099228      0.248126   
6     100_MIDDLE        422.370389    0.230859     0.239648      0.219960   
7     200_MIDDLE        414.859848    0.207026     0.157393      0.165939   
8      100_CARRY        381.774365    0.153613     0.198248      0.243337   
9     200_JUNGLE        402.083862    0.233919     0.171119      0.306281   
10   200_SUPPORT        338.770297    0.108692     0.194199      0.571228   
11    100_MIDDLE        428.481377    0.270147     0.166586      0.338738   

In [797]:
all_stats_matchs_pd = all_stats_matchs_pd.drop(['id','number_of_players'],axis=1)

labels ['id' 'number_of_players'] not contained in axis
Traceback (most recent call last):
  File "/usr/bin/anaconda/lib/python2.7/site-packages/pandas/core/generic.py", line 2050, in drop
    new_axis = axis.drop(labels, errors=errors)
  File "/usr/bin/anaconda/lib/python2.7/site-packages/pandas/core/indexes/base.py", line 3575, in drop
    labels[mask])
ValueError: labels ['id' 'number_of_players'] not contained in axis



In [799]:
matchs_group = all_stats_matchs_pd.groupby("match_id").groups

In [802]:
print("there are {} legit matches for final matrix".format(len(matchs_group)))

there are 35 legit matches for final matrix

In [803]:
one_match = all_stats_matchs_pd.iloc[matchs_group['3207612018']]

In [842]:
feat_list = list(one_match.columns) #list of features
feat_list 

['final_role', 'avg(gold_earned)', 'avg(kills)', 'avg(deaths)', 'avg(assists)', 'avg(visionScore)', 'avg(totalDamageDealt)', 'avg(totalDamageTaken)', 'avg(totalMinionsKilled)', 'avg(totalTimeCrowdControlDealt)', 'summonerID', 'championId', 'points', 'name', 'armorperlevel', 'attackdamageperlevel', 'attackspeed', 'attackrange', 'spellblockperlevel', 'hpperlevel', 'match_id', 'label']

In [844]:
features_matrix_list = []
for group_key in matchs_group.keys():
    one_match = all_stats_matchs_pd.iloc[matchs_group[group_key]]
    feat_count = 2
    match_dict = {} #create a dict for 1 match 
    match_dict['label'] = int(one_match.iloc[0]['label'])
    match_dict['match_id'] = str(one_match.iloc[0]['match_id'])
    for user in one_match.iterrows():
        player = user[1]
        role = player['final_role']
        processed = ['final_role','label','match_id']
        string_feat = ['summonerID','championId','name']
        int_feat = ['label']
        for feat in feat_list:
            if feat not in processed:
                if feat in string_feat:
                    val = str(player[feat])
                else:
                    val = float(player[feat])
                match_dict["{}_{}".format(role,feat)] = val
                feat_count += 1
            
    if len(match_dict) == 192:
        features_matrix_list.append(match_dict)

In [845]:
print("after cleaning data, we have {} matches left".format(len(features_matrix_list)))

after cleaning data, we have 21 matches left

# Final feature matrix

In [849]:
myJson = sc.parallelize(features_matrix_list)
features_matrix = sqlContext.read.json(myJson) #change list of dict to pyspark dataframe

In [853]:
features_matrix.count()

21

## save feature matrix to CosmosDB

In [855]:
writeConfig = {
    "Endpoint": "https://lolpredict.documents.azure.com:443/",
    "Masterkey": "UcLcT6bvFUsk677XfibNhJIIqiAstRkaWWrqTJaQOCmVqx0IN0J1ufYdpbsarowutD6nMSKaaLS1NqcD0QDIdQ==",
    "Database": "league_extraction",
    "Collection": "final_features_matrix",
    "Upsert": "true"
}

# Write to Cosmos DB from the flights DataFrame
features_matrix.write.format("com.microsoft.azure.cosmosdb.spark").options(
    **writeConfig).save()